# Consolidated Multi-Model Analysis - Complete Pipeline

This notebook is **completely self-contained** and executes **all 9 analytical models** in a single pipeline.

**Key Features:**
- ✅ Starts from raw CSV (no dependencies)
- ✅ Full preprocessing pipeline (load → validate → transform)
- ✅ Executes all 9 models using ExecutionOrchestrator
- ✅ Handles model dependencies automatically
- ✅ Exports all results to single Excel file
- ✅ Saves context state for downstream notebooks

**Pipeline Flow:**
```
Raw CSV → Preprocessing → transactions_enriched
                                ↓
                          transactions (Model 1)
                                ↓
                        transactions_filters
                                ↓
    ┌───────────────────────────┼───────────────────────────┐
    ↓                           ↓                           ↓
  daily                   product_daily              customer_daily
    ↓                           ↓                           ↓
  daily_attrs            product_month              customer_profile
    ↓
 ┌──┴──────┐
 ↓         ↓
daily_hour weekly
           ↓
         monthly
```

**Models Executed (9 total):**
1. **transactions** - Transaction-level enrichment (43 filters, 0 attributes)
2. **daily** - Daily business metrics (23 attributes)
3. **daily_hour** - Hourly patterns with daily comparisons (3 filters + 23 attributes)
4. **product_daily** - Product-day performance (8 attributes)
5. **customer_daily** - Customer-day activity (9 attributes)
6. **weekly** - Weekly KPIs (8 attributes)
7. **monthly** - Monthly trends (7 attributes)
8. **product_month** - Product-month lifecycle (6 attributes)
9. **customer_profile** - Customer profiles (7 attributes)

**Input Required:** Single CSV file path in configuration below

## 0. Project Root Setup

In [1]:
# Auto-detect project root and add to Python path
import os
import sys
from pathlib import Path

# Get the project root (2 levels up from notebooks/from_store)
notebook_dir = Path.cwd() if '__file__' not in globals() else Path(__file__).parent
project_root = notebook_dir.parent.parent

# Change to project root
os.chdir(project_root)

# Add project root to Python path if not already there
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

print(f"Working directory: {os.getcwd()}")
print(f"Project root: {project_root}")

Working directory: c:\Projects\play\khujta_ai_business
Project root: c:\Projects\play\khujta_ai_business


## 1. Configuration

**IMPORTANT:** Set your input CSV file path and data schema here.

The `data_schema` maps source columns (which can vary) to standardized internal names.

In [2]:
# Base configuration
base_cfg = {
    # ============================================================
    # INPUT FILE - Change this to your CSV path
    # ============================================================
    'input_file': 'data/tests/transactions/test_transactions_30days.csv',
    
    # ============================================================
    # METADATA
    # ============================================================
    'client': 'consolidated_analysis',
    'analysis_dt': '2025-11-11',
    'log_level': 'INFO',
    'fidx_config': {'type': 'local', 'path': 'feature_store'},
    
    # ============================================================
    # DEFAULT FORMATS (applied to all columns unless overridden)
    # ============================================================
    'default_formats': {
        'date': '%m/%d/%Y %H:%M',
        'float': {'thousands': '.', 'decimal': ','},
        'int': {'thousands': '.', 'decimal': ','},
    },
    
    # ============================================================
    # DATA SCHEMA - Column Mappings
    # Maps source CSV columns → standardized internal names + types
    # ============================================================
    'data_schema': {
        'in_dt': {
            'source_column': 'in_dt',
            'dtype': 'date'
        },
        'in_trans_id': {
            'source_column': 'in_trans_id',
            'dtype': 'str'
        },
        'in_trans_type': {
            'source_column': 'in_trans_type',
            'dtype': 'str'
        },
        'in_customer_id': {
            'source_column': 'in_customer_id',
            'dtype': 'str'
        },
        'in_product_id': {
            'source_column': 'in_product_id',
            'dtype': 'str'
        },
        'in_description': {
            'source_column': 'in_description',
            'dtype': 'str'
        },
        'in_category': {
            'source_column': 'in_category',
            'dtype': 'str'
        },
        'in_unit_type': {
            'source_column': 'in_unit_type',
            'dtype': 'str'
        },
        'in_stock': {
            'source_column': 'in_stock',
            'dtype': 'float'
        },
        'in_quantity': {
            'source_column': 'in_quantity',
            'dtype': 'float'
        },
        'in_cost_unit': {
            'source_column': 'in_cost_unit',
            'dtype': 'float'
        },
        'in_cost_total': {
            'source_column': 'in_cost_total',
            'dtype': 'float'
        },
        'in_price_unit': {
            'source_column': 'in_price_unit',
            'dtype': 'float'
        },
        'in_price_total': {
            'source_column': 'in_price_total',
            'dtype': 'float'
        },
        'in_discount_total': {
            'source_column': 'in_discount_total',
            'dtype': 'float'
        },
        'in_commission_total': {
            'source_column': 'in_commission_total',
            'dtype': 'float'
        },
        'in_margin': {
            'source_column': 'in_margin',
            'dtype': 'float'
        }
    }
}

print(f"✓ Configuration loaded")
print(f"  Input file: {base_cfg['input_file']}")
print(f"  Client: {base_cfg['client']}")
print(f"  Columns mapped: {len(base_cfg['data_schema'])}")

✓ Configuration loaded
  Input file: data/tests/transactions/test_transactions_30days.csv
  Client: consolidated_analysis
  Columns mapped: 17


## 2. Imports + Initialize Context

In [3]:
import pandas as pd
import numpy as np
from datetime import datetime

# v2.0 Refactored imports
from src.utils.logger import setup_logging, get_logger
from src.core.context import GabedaContext
from src.core.persistence import save_context_state
from src.core.constants import *
from src.preprocessing.loaders import DataLoader
from src.preprocessing.validators import DataValidator
from src.preprocessing.schema import SchemaProcessor
from src.preprocessing.synthetic import SyntheticEnricher
from src.features.store import FeatureStore
from src.features.resolver import DependencyResolver
from src.features.detector import FeatureTypeDetector
from src.features.analyzer import FeatureAnalyzer
from src.execution.calculator import FeatureCalculator
from src.execution.groupby import GroupByProcessor
from src.execution.executor import ModelExecutor
from src.execution.orchestrator import ExecutionOrchestrator
from src.export.excel import ExcelExporter

# Setup logging
setup_logging(
    log_level=base_cfg.get('log_level', 'INFO'),
    config={'client': base_cfg.get('client', 'unknown_client')}
)
logger = get_logger(__name__)

# Initialize NEW context (creates fresh context, not loading from state)
ctx = GabedaContext(base_cfg)

print(f"✓ All imports successful")
print(f"✓ Logging initialized")
print(f"✓ Context created: {ctx.run_id}")

📝 Run instance ID: consolidated_analysis_20251022_173402 - Logging [INFO] to: logs\consolidated_analysis_20251022_173402.log
✓ All imports successful
✓ Logging initialized
✓ Context created: consolidated_analysis_20251022_173402


## 3. Load & Preprocess Data

Complete preprocessing pipeline:
1. Load raw CSV
2. Validate schema and data quality
3. Process schema (column mapping + type conversion)
4. Synthetic enrichment (auto-inference)
5. Row-level validation
6. Store datasets in context

In [4]:
# ============================================================
# STEP 1: Load raw data
# ============================================================
loader = DataLoader()
raw_data = loader.load_csv(base_cfg['input_file'])
print(f"✓ Loaded raw data: {raw_data.shape}")

# ============================================================
# STEP 2: Comprehensive validation
# ============================================================
validator = DataValidator()
required_cols = [spec['source_column'] for spec in base_cfg['data_schema'].values()]

validation = validator.validate_all(
    df=raw_data,
    required_cols=required_cols,
    data_schema=base_cfg['data_schema'],
    default_formats=base_cfg['default_formats']
)

if not validation.is_valid:
    print("❌ Validation FAILED!")
    for error in validation.errors:
        print(f"  • {error}")
    raise ValueError("Fix validation errors before proceeding")

if validation.warnings:
    print(f"⚠️  Warnings ({len(validation.warnings)}):")
    for warning in validation.warnings[:5]:
        print(f"  • {warning}")

print("✓ Validation passed")

# ============================================================
# STEP 3: Process schema (column mapping + type conversion)
# ============================================================
schema_processor = SchemaProcessor()
preprocessed_df = schema_processor.process_schema(raw_data, base_cfg).df
print(f"✓ Schema processed: {preprocessed_df.shape}")

# ============================================================
# STEP 4: Synthetic enrichment
# ============================================================
enricher = SyntheticEnricher(synthetic_model_name='synthetic')
preprocessed_df = enricher.enrich(data=preprocessed_df)
print(f"✓ Data enriched: {preprocessed_df.shape}")

# ============================================================
# STEP 5: Row-level validation
# ============================================================
transactions_enriched, reject_result = validator.validate_row_level_required_fields(
    df=preprocessed_df,
    data_schema=base_cfg['data_schema'],
    save_to_file=False
)

# ============================================================
# STEP 6: Store datasets in context
# ============================================================
ctx.set_dataset('transactions_raw', raw_data)
ctx.set_dataset('transactions_enriched', transactions_enriched)
if reject_result.rejected_rows is not None:
    ctx.set_dataset('transactions_rejected', reject_result.rejected_rows)

# ============================================================
# STEP 7: Report data quality summary
# ============================================================
if reject_result.rejected_rows is not None:
    print(f"\n📊 Data Quality Summary:")
    print(f"  Total rows: {len(preprocessed_df)}")
    print(f"  Clean rows: {len(transactions_enriched)}")
    print(f"  Rejected rows: {len(reject_result.rejected_rows)}")
    print(f"  Rejection rate: {len(reject_result.rejected_rows) / len(preprocessed_df) * 100:.2f}%")
else:
    print(f"\n✓ All {len(transactions_enriched)} rows passed validation")

print(f"\n✓ Preprocessing complete!")
print(f"  Datasets created: {ctx.list_datasets()}")

✓ Loaded raw data: (609, 17)
✓ Validation passed
✓ Schema processed: (609, 17)
✓ Data enriched: (609, 17)

✓ All 609 rows passed validation

✓ Preprocessing complete!
  Datasets created: ['transactions_raw', 'transactions_enriched']


## 4. Configure Pipeline

Define all 9 models and their execution order.

**Execution Order** (respects dependencies):
1. transactions → creates transactions_filters
2. daily → creates daily_attrs (needed by daily_hour and weekly)
3. daily_hour → uses daily_attrs for cross-model comparisons
4. product_daily → independent
5. customer_daily → independent
6. weekly → uses daily_attrs for calculations
7. monthly → independent
8. product_month → independent
9. customer_profile → independent

In [5]:
# Initialize feature store
feature_store = FeatureStore()

# Define pipeline execution order - ALL 9 MODELS
pipeline_models = [
    {
        'name': 'transactions',
        'description': 'Transaction-level enrichment (43 filters)',
        'input_dataset': 'transactions_enriched'
    },
    {
        'name': 'daily',
        'description': 'Daily business metrics (23 attributes)',
        'input_dataset': 'transactions_filters'
    },
    {
        'name': 'daily_hour',
        'description': 'Hourly patterns with daily comparisons (3 filters + 23 attributes)',
        'input_dataset': 'transactions_filters',
        'requires_external': ['daily_attrs']
    },
    {
        'name': 'product_daily',
        'description': 'Product-day performance (8 attributes)',
        'input_dataset': 'transactions_filters'
    },
    {
        'name': 'customer_daily',
        'description': 'Customer-day activity (9 attributes)',
        'input_dataset': 'transactions_filters'
    },
    {
        'name': 'weekly',
        'description': 'Weekly KPIs (8 attributes)',
        'input_dataset': 'transactions_filters',
        'requires_external': ['daily_attrs']
    },
    {
        'name': 'monthly',
        'description': 'Monthly trends (7 attributes)',
        'input_dataset': 'transactions_filters'
    },
    {
        'name': 'product_month',
        'description': 'Product-month lifecycle (6 attributes)',
        'input_dataset': 'transactions_filters'
    },
    {
        'name': 'customer_profile',
        'description': 'Customer profiles (7 attributes)',
        'input_dataset': 'transactions_filters'
    }
]

print(f"✓ Pipeline configured with {len(pipeline_models)} models:")
for i, model_spec in enumerate(pipeline_models, 1):
    ext_info = f" (uses {', '.join(model_spec.get('requires_external', []))})" if model_spec.get('requires_external') else ""
    print(f"  {i}. {model_spec['name']}: {model_spec['description']}{ext_info}")

✓ Pipeline configured with 9 models:
  1. transactions: Transaction-level enrichment (43 filters)
  2. daily: Daily business metrics (23 attributes)
  3. daily_hour: Hourly patterns with daily comparisons (3 filters + 23 attributes) (uses daily_attrs)
  4. product_daily: Product-day performance (8 attributes)
  5. customer_daily: Customer-day activity (9 attributes)
  6. weekly: Weekly KPIs (8 attributes) (uses daily_attrs)
  7. monthly: Monthly trends (7 attributes)
  8. product_month: Product-month lifecycle (6 attributes)
  9. customer_profile: Customer profiles (7 attributes)


## 5. Load Models from Feature Store

Load each model's configuration and features from feature_store/.

In [6]:
# Load all models and prepare for execution
loaded_models = []

print("Loading models from feature store...\n")

for model_spec in pipeline_models:
    model_name = model_spec['name']
    
    try:
        # Load model from feature store
        model = feature_store.load_model(model_name)
        cfg_model = model['master_cfg']
        
        # LOG 1: How many features were loaded from feature_store for this model
        print(f"✓ {model_name}")
        print(f"  - Model: {model_name}")
        print(f"  - Features loaded from store: {len(cfg_model['features'])}")
        print(f"  - Feature names: {list(cfg_model['features'].keys())[:5]}{'...' if len(cfg_model['features']) > 5 else ''}")
        print(f"  - Execution sequence: {len(cfg_model['exec_seq'])} steps")
        print(f"  - Group by: {cfg_model.get('group_by', 'None')}")
        print(f"  - Input dataset: {model_spec['input_dataset']}")
        
        # Check for external data requirements
        if cfg_model.get('ext_cols'):
            print(f"  - External data required: {list(cfg_model['ext_cols'].keys())}")
        print()
        
        # Add to loaded models
        loaded_models.append({
            'name': model_name,
            'config': cfg_model,
            'input_dataset': model_spec['input_dataset']
        })
        
    except Exception as e:
        print(f"✗ Failed to load {model_name}: {e}")
        import traceback
        traceback.print_exc()
        raise

print(f"\n✓ Successfully loaded {len(loaded_models)}/{len(pipeline_models)} models")
print(f"  Total features across all models: {sum(len(m['config']['features']) for m in loaded_models)}")

Loading models from feature store...

✓ transactions
  - Model: transactions
  - Features loaded from store: 51
  - Feature names: ['category', 'commission_total', 'cost_total', 'cost_unit', 'customer_id']...
  - Execution sequence: 43 steps
  - Group by: None
  - Input dataset: transactions_enriched
  - External data required: ['dict', 'list']

✓ daily
  - Model: daily
  - Features loaded from store: 23
  - Feature names: ['cost_total_mean', 'cost_total_median', 'cost_total_sum', 'customer_id_count', 'margin_total_mean']...
  - Execution sequence: 23 steps
  - Group by: ['dt_date']
  - Input dataset: transactions_filters
  - External data required: ['dict', 'list']

✓ daily_hour
  - Model: daily_hour
  - Features loaded from store: 31
  - Feature names: ['cost_total_mean', 'cost_total_median', 'cost_total_sum', 'customer_id_count', 'customer_id_vs_daily_count_pct']...
  - Execution sequence: 26 steps
  - Group by: ['dt_date', 'hour']
  - Input dataset: transactions_filters
  - Externa

## 6. Execute Pipeline

Use `ExecutionOrchestrator` to execute all 9 models in sequence with automatic dependency handling.

In [7]:
# Initialize execution components
detector = FeatureTypeDetector()
analyzer = FeatureAnalyzer(feature_store, detector)
calculator = FeatureCalculator()
groupby_processor = GroupByProcessor(calculator, detector)
executor = ModelExecutor(analyzer, groupby_processor, context=ctx)
orchestrator = ExecutionOrchestrator(executor, ctx)

print("✓ Execution components initialized")
print(f"\nStarting pipeline execution at {datetime.now().strftime('%H:%M:%S')}...\n")
print("="*80)

# Execute pipeline
start_time = datetime.now()

results = orchestrator.execute_pipeline(
    models=loaded_models,
    initial_dataset_name='transactions_enriched'
)

end_time = datetime.now()
elapsed = (end_time - start_time).total_seconds()

print("="*80)
print(f"\n✓ Pipeline execution complete!")
print(f"  - Models executed: {len(results)}")
print(f"  - Elapsed time: {elapsed:.2f} seconds")
print(f"  - Completed at: {end_time.strftime('%H:%M:%S')}")

✓ Execution components initialized

Starting pipeline execution at 17:34:02...


✓ Pipeline execution complete!
  - Models executed: 9
  - Elapsed time: 4.72 seconds
  - Completed at: 17:34:07


## 7. View Pipeline Results

Summary of all executed models and their outputs.

In [8]:
print("\n" + "="*80)
print("PIPELINE RESULTS SUMMARY")
print("="*80)

print(f"\nTotal Datasets in Context: {len(ctx.datasets)}")
print(f"Total Models Executed: {len(ctx.models)}")

print("\n" + "-"*80)
print("MODEL OUTPUTS")
print("-"*80)

for i, model_name in enumerate(ctx.list_models(), 1):
    model_info = ctx.models[model_name]
    input_name = model_info.get('input_dataset_name', 'N/A')
    
    # Get output shapes
    filters = ctx.get_model_filters(model_name)
    attrs = ctx.get_model_attrs(model_name)
    
    filters_shape = filters.shape if filters is not None else 'None'
    attrs_shape = attrs.shape if attrs is not None else 'None'
    
    print(f"\n{i}. {model_name.upper()}")
    print(f"   Input:  {input_name}")
    print(f"   Filters: {filters_shape}")
    print(f"   Attrs:   {attrs_shape}")

print("\n" + "="*80)


PIPELINE RESULTS SUMMARY

Total Datasets in Context: 19
Total Models Executed: 9

--------------------------------------------------------------------------------
MODEL OUTPUTS
--------------------------------------------------------------------------------

1. TRANSACTIONS
   Input:  transactions_enriched
   Filters: (609, 59)
   Attrs:   None

2. DAILY
   Input:  transactions_filters
   Filters: (609, 58)
   Attrs:   (30, 24)

3. DAILY_HOUR
   Input:  transactions_filters
   Filters: (609, 83)
   Attrs:   (301, 25)

4. PRODUCT_DAILY
   Input:  transactions_filters
   Filters: (609, 57)
   Attrs:   (266, 10)

5. CUSTOMER_DAILY
   Input:  transactions_filters
   Filters: (609, 57)
   Attrs:   (327, 11)

6. WEEKLY
   Input:  transactions_filters
   Filters: (609, 83)
   Attrs:   (5, 7)

7. MONTHLY
   Input:  transactions_filters
   Filters: (609, 61)
   Attrs:   (1, 5)

8. PRODUCT_MONTH
   Input:  transactions_filters
   Filters: (609, 58)
   Attrs:   (10, 7)

9. CUSTOMER_PROFILE
   In

## 8. Preview Key Results

Quick look at some of the key analytical outputs.

### 8.1 Daily Business Metrics

In [9]:
daily_attrs = ctx.get_model_attrs('daily')
print(f"Daily Metrics (n={len(daily_attrs)} days):")
display(daily_attrs.head(10))

print("\nDaily Revenue Summary:")
display(daily_attrs[['price_total_sum', 'trans_id_count', 'customer_id_count']].describe())

Daily Metrics (n=30 days):


,dt_date,year,quarter,month,weekofyear,customer_id_count,product_id_count,trans_id_count,quantity_sum,quantity_mean,...,cost_total_sum,cost_total_mean,cost_total_median,transaction_mean,margin_total_sum,margin_total_mean,margin_total_median,margin_total_pct_min,margin_total_pct_mean,margin_total_pct_median
0,20251001,2025,4,10,40,14,9,22,71,3.23,...,1085435.0,49337.95,47665.5,79122.77,678405.0,32305.000000,31241.0,25.330000,39.020000,34.209999
1,20251002,2025,4,10,40,12,10,28,94,3.36,...,1429106.0,51039.50,33156.5,87682.32,1042216.0,38600.601562,25959.0,19.250000,41.799999,33.389999
2,20251003,2025,4,10,40,13,9,29,120,4.14,...,1876081.0,64692.45,40738.0,94232.76,941812.0,34881.921875,21269.0,20.790001,34.910000,31.240000
3,20251004,2025,4,10,40,10,10,23,98,4.26,...,1453574.0,63198.87,37963.0,99762.70,876471.0,41736.718750,28923.0,20.820000,39.230000,34.790001
4,20251005,2025,4,10,40,8,8,16,72,4.50,...,941393.0,58837.06,37837.5,85382.94,481244.0,34374.570312,19076.5,16.010000,35.250000,33.180000
5,20251006,2025,4,10,41,9,8,16,46,2.88,...,895889.0,55993.06,38932.5,37444.38,162905.0,14809.549805,12795.0,17.750000,32.369999,33.000000
6,20251007,2025,4,10,41,10,7,17,53,3.12,...,751192.0,44187.76,37043.0,70132.00,458038.0,28627.380859,22994.5,21.930000,38.980000,30.330000
7,20251008,2025,4,10,41,10,8,18,51,2.83,...,615279.0,34182.17,28343.5,51091.56,319478.0,18792.820312,17325.0,24.400000,36.810001,33.730000
8,20251009,2025,4,10,41,13,10,26,85,3.27,...,1430065.0,55002.50,32367.0,81660.31,735411.0,30642.119141,27034.0,15.630000,36.259998,31.070000
9,20251010,2025,4,10,41,12,10,26,89,3.42,...,1280582.0,49253.15,34648.5,83047.04,1028579.0,46753.589844,22008.0,18.290001,40.330002,33.080002



Daily Revenue Summary:


,price_total_sum,trans_id_count,customer_id_count
count,3.000000e+01,30.000000,30.000000
mean,1.783194e+06,20.300000,10.900000
std,7.230171e+05,6.018076,2.171127
min,3.526590e+05,8.000000,6.000000
25%,1.235715e+06,17.000000,9.250000
50%,1.838787e+06,19.000000,11.000000
75%,2.156682e+06,25.250000,12.750000
max,3.413690e+06,35.000000,14.000000


### 8.2 Customer Profiles

In [10]:
customer_profile_attrs = ctx.get_model_attrs('customer_profile')
print(f"Customer Profiles (n={len(customer_profile_attrs)} customers):")
display(customer_profile_attrs.head(10))

print("\nTop 5 Customers by Spending:")
top_customers = customer_profile_attrs.nlargest(5, 'month_total_spent')[[
    'customer_id', 'month_total_spent', 'month_transaction_count', 'average_order_value'
]]
display(top_customers)

Customer Profiles (n=15 customers):


,customer_id,month_total_spent,month_transaction_count,month_items_purchased,average_order_value,favorite_products,preferred_shopping_day,preferred_shopping_time
0,CLIENT1,3835713.0,43,176,89202.63,"PROD2,PROD9,PROD8",Friday,Afternoon
1,CLIENT10,2642342.0,33,116,80070.97,"PROD9,PROD7,PROD1",Saturday,Afternoon
2,CLIENT11,3263459.0,42,175,77701.40,"PROD9,PROD8,PROD10",Thursday,Afternoon
3,CLIENT12,4057118.0,59,184,68764.71,"PROD1,PROD3,PROD10",Thursday,Afternoon
4,CLIENT13,3407417.0,31,144,109916.68,"PROD8,PROD3,PROD6",Friday,Afternoon
5,CLIENT14,4637137.0,50,198,92742.74,"PROD9,PROD4,PROD10",Wednesday,Afternoon
6,CLIENT15,3423471.0,41,166,83499.29,"PROD10,PROD7,PROD1",Thursday,Afternoon
7,CLIENT2,3425998.0,44,151,77863.59,"PROD2,PROD6,PROD9",Thursday,Afternoon
8,CLIENT3,4731427.0,40,188,118285.68,"PROD7,PROD4,PROD8",Thursday,Afternoon
9,CLIENT4,2597804.0,34,106,76406.00,"PROD10,PROD3,PROD1",Wednesday,Afternoon



Top 5 Customers by Spending:


,customer_id,month_total_spent,month_transaction_count,average_order_value
13,CLIENT8,4782574.0,44,108694.86
8,CLIENT3,4731427.0,40,118285.68
5,CLIENT14,4637137.0,50,92742.74
3,CLIENT12,4057118.0,59,68764.71
0,CLIENT1,3835713.0,43,89202.63


## 9. Export All Models to Excel

Export all models to a single Excel file with separate tabs for each model's filters and attributes.

In [11]:
# Initialize exporter
exporter = ExcelExporter(ctx)

# Export all models
output_file = 'outputs/consolidated_all_models_export.xlsx'
result = exporter.export_all_models(
    output_file,
    include_unique_inputs=True
)

print("\n" + "="*80)
print("EXCEL EXPORT COMPLETE")
print("="*80)
print(f"\n✓ File saved: {output_file}")
print(f"\nExpected Excel Structure:")
print(f"  - Tab 1: transactions_enriched (input)")

tab_num = 2
for model_name in ctx.list_models():
    filters = ctx.get_model_filters(model_name)
    attrs = ctx.get_model_attrs(model_name)
    
    if filters is not None:
        print(f"  - Tab {tab_num}: {model_name}_filters")
        tab_num += 1
    if attrs is not None:
        print(f"  - Tab {tab_num}: {model_name}_attrs")
        tab_num += 1

print(f"\nTotal tabs: {tab_num - 1}")
print("\nNote: All tabs have autofilters enabled!")


EXCEL EXPORT COMPLETE

✓ File saved: outputs/consolidated_all_models_export.xlsx

Expected Excel Structure:
  - Tab 1: transactions_enriched (input)
  - Tab 2: transactions_filters
  - Tab 3: daily_filters
  - Tab 4: daily_attrs
  - Tab 5: daily_hour_filters
  - Tab 6: daily_hour_attrs
  - Tab 7: product_daily_filters
  - Tab 8: product_daily_attrs
  - Tab 9: customer_daily_filters
  - Tab 10: customer_daily_attrs
  - Tab 11: weekly_filters
  - Tab 12: weekly_attrs
  - Tab 13: monthly_filters
  - Tab 14: monthly_attrs
  - Tab 15: product_month_filters
  - Tab 16: product_month_attrs
  - Tab 17: customer_profile_filters
  - Tab 18: customer_profile_attrs

Total tabs: 18

Note: All tabs have autofilters enabled!


## 10. Save Context State

Save the context with all model outputs for future use by downstream notebooks.

In [12]:
# Save context state
# IMPORTANT: Always create fresh context when starting from CSV (like 00_transactions.ipynb)
# This is the FIRST model (transactions) in the pipeline, so we need a clean context
state_dir = save_context_state(ctx=ctx, base_cfg=base_cfg, reuse_existing=False)

print(f"✓ Context state saved: {state_dir}")
print(f"  - Total datasets: {len(ctx.datasets)}")
print(f"  - Total models: {len(ctx.models)}")
print(f"\nTo load this state in another notebook:")
print(f"  from src.core.persistence import load_context_state")
print(f"  ctx, base_cfg = load_context_state('{state_dir}')")

✓ Context state saved: data\context_states\consolidated_analysis_20251022_173402
  - Total datasets: 19
  - Total models: 9

To load this state in another notebook:
  from src.core.persistence import load_context_state
  ctx, base_cfg = load_context_state('data\context_states\consolidated_analysis_20251022_173402')


## Summary

### What This Notebook Accomplished

✅ **Self-Contained Execution** - Started from raw CSV, no dependencies

✅ **Complete Preprocessing** - Load → Validate → Transform → Store

✅ **9 Models Executed** - Complete business intelligence pipeline

✅ **Pipeline Orchestration** - ExecutionOrchestrator handled all dependencies

✅ **Single Excel Export** - All results in one file (~20 tabs)

✅ **Context Saved** - State available for downstream analysis

### Models Executed

| Model | Type | Group By | Key Metrics |
|-------|------|----------|-------------|
| transactions | Filters | None | 43 enrichment features |
| daily | Attributes | dt_date | 23 daily KPIs |
| daily_hour | Mixed | dt_date, hour | 3 filters + 23 attributes |
| product_daily | Attributes | product_id, dt_date | 8 product metrics |
| customer_daily | Attributes | customer_id, dt_date | 9 customer metrics |
| weekly | Attributes | year, weekofyear | 8 weekly KPIs |
| monthly | Attributes | year, month | 7 monthly trends |
| product_month | Attributes | product_id, year, month | 6 lifecycle metrics |
| customer_profile | Attributes | customer_id | 7 profile features |

### Next Steps

1. **Visualization** - Create dashboard from Excel export
2. **Automation** - Schedule notebook for daily/weekly runs
3. **Analysis** - Deep dive into specific metrics
4. **Expansion** - Add more models to feature_store